In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set plotting style for better aesthetics
sns.set_theme(style="whitegrid")

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# --- Configuration ---
INPUT_FILE = "../data/raw/Database_CPT_PremstallerGeotechnik/CPT_PremstallerGeotechnik_revised.csv" 
# -------------------

# Load the dataset
try:
    df = pd.read_csv(INPUT_FILE)
    print("File loaded successfully!")
except FileNotFoundError:
    print(f"Error: File not found at '{INPUT_FILE}'. Please check the path.")

# Display the first few rows to understand the format
print("First 5 rows of the dataset:")
display(df.head())

# Get a concise summary of the dataframe
print("\nDataset Info:")
df.info()

# Get summary statistics for numerical columns
print("\nSummary Statistics:")
display(df.describe())

In [ ]:
# Analyze the number of unique CPT tests
unique_cpts = df['ID'].unique()
print(f"Total number of unique CPTs: {len(unique_cpts)}")

# Analyze the depth and measurement counts for each CPT
cpt_lengths = df.groupby('ID').size()
print("\nDistribution of CPT lengths (number of measurements):")
display(cpt_lengths.describe())

In [ ]:
# Define key feature columns
feature_cols = ['qc (MPa)', 'fs (kPa)', 'u2 (kPa)']

# Plot histograms for each key feature
plt.figure(figsize=(18, 5))
for i, col in enumerate(feature_cols, 1):
    plt.subplot(1, 3, i)
    sns.histplot(df[col].dropna(), kde=True, bins=50)
    plt.title(f'Distribution of {col}')
    plt.yscale('log') # Use log scale due to high skewness
plt.tight_layout()
plt.show()

# Plot boxplots to see outliers more clearly
plt.figure(figsize=(18, 5))
for i, col in enumerate(feature_cols, 1):
    plt.subplot(1, 3, i)
    sns.boxplot(y=df[col].dropna())
    plt.title(f'Boxplot of {col}')
    plt.yscale('log') # Use log scale to better visualize the distribution
plt.tight_layout()
plt.show()

In [ ]:
def plot_cpt_profile(df, cpt_id):
    """Plots qc, fs, and u2 against depth for a given CPT ID."""
    cpt_data = df[df['ID'] == cpt_id].sort_values(by='Depth (m)')
    
    if cpt_data.empty:
        print(f"No data found for CPT ID: {cpt_id}")
        return

    fig, axes = plt.subplots(1, 3, figsize=(15, 8), sharey=True)
    fig.suptitle(f'CPT Profile for ID: {cpt_id}', fontsize=16)

    # Plot Cone Resistance (qc)
    axes[0].plot(cpt_data['qc (MPa)'], cpt_data['Depth (m)'], color='b')
    axes[0].set_xlabel('Cone Resistance, qc (MPa)')
    axes[0].set_ylabel('Depth (m)')
    axes[0].grid(True)
    
    # Plot Sleeve Friction (fs)
    axes[1].plot(cpt_data['fs (kPa)'], cpt_data['Depth (m)'], color='r')
    axes[1].set_xlabel('Sleeve Friction, fs (kPa)')
    axes[1].grid(True)
    
    # Plot Pore Pressure (u2)
    axes[2].plot(cpt_data['u2 (kPa)'], cpt_data['Depth (m)'], color='g')
    axes[2].set_xlabel('Pore Pressure, u2 (kPa)')
    axes[2].grid(True)
    
    # Invert y-axis to have depth increase downwards
    for ax in axes:
        ax.invert_yaxis()
        
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# --- Plot a few example profiles ---
# Select a few IDs to visualize from the `unique_cpts` array
example_ids = unique_cpts[:3] # Plot the first 3 CPTs in the dataset

for test_id in example_ids:
    plot_cpt_profile(df, test_id)

In [ ]:
# Select a subset of interesting numerical columns for correlation analysis
corr_cols = [
    'Depth (m)', 'qc (MPa)', 'fs (kPa)', 'u2 (kPa)', 'qt (MPa)', 
    'Rf (%)', 'γ (kN/m³)', "σ',v (kPa)", 'Qtn (-)', 'Fr (%)', 'Ic (-)'
]

# Ensure all selected columns exist in the DataFrame
corr_cols_exist = [col for col in corr_cols if col in df.columns]

# Calculate the correlation matrix
correlation_matrix = df[corr_cols_exist].corr()

# Plot the heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', linewidths=.5)
plt.title('Correlation Matrix of CPT Parameters')
plt.show()

In [ ]:
# --- Configuration ---
SOIL_CLASS_COLUMN = 'EN_ISO_14688_classes'
# -------------------

# First, check for missing values in the soil class column
missing_classes = df[SOIL_CLASS_COLUMN].isnull().sum()
total_rows = len(df)
print(f"Missing soil class values: {missing_classes} ({missing_classes / total_rows:.2%})")

# Get the count of each unique soil class
class_counts = df[SOIL_CLASS_COLUMN].dropna().value_counts()

print("\nCounts of each soil class:")
print(class_counts)

# Visualize the distribution of soil classes with a bar chart
plt.figure(figsize=(12, 7))
sns.countplot(y=df[SOIL_CLASS_COLUMN].dropna(), order=class_counts.index)
plt.title('Frequency of Soil Classes (EN_ISO_14688)')
plt.xlabel('Count')
plt.ylabel('Soil Class')
plt.xscale('log') # Use a log scale if counts are highly variable
plt.grid(axis='x')
plt.show()

In [ ]:
# Create depth bins (e.g., every 5 meters)
max_depth = df['Depth (m)'].max()
depth_bins = np.arange(0, max_depth + 5, 5)
df['depth_bin'] = pd.cut(df['Depth (m)'], bins=depth_bins, right=False)

# Create a crosstab to count soil classes within each depth bin
class_depth_distribution = pd.crosstab(
    df['depth_bin'],
    df[SOIL_CLASS_COLUMN]
)

# Normalize the counts to get proportions
class_depth_proportion = class_depth_distribution.div(class_depth_distribution.sum(axis=1), axis=0)

# Plot the normalized stacked bar chart
class_depth_proportion.plot(kind='bar', stacked=True, figsize=(15, 8), colormap='viridis')
plt.title('Proportional Distribution of Soil Classes with Depth')
plt.xlabel('Depth Range (m)')
plt.ylabel('Proportion of Soil Class')
plt.xticks(rotation=45)
plt.legend(title='Soil Class', bbox_to_anchor=(1.02, 1), loc='upper left')
plt.grid(axis='y')
plt.tight_layout()
plt.show()

In [ ]:
# --- Configuration ---
SOIL_CLASS_COLUMN = 'EN_ISO_1_4688_classes'
NUMERICAL_COLS = ['qc (MPa)', 'fs (kPa)', 'u2 (kPa)']
# -------------------

# Create a clean dataframe for this analysis, dropping rows where key values are missing
analysis_df = df[[SOIL_CLASS_COLUMN] + NUMERICAL_COLS].dropna()

# Get the order of soil classes from most to least frequent for better plotting
class_order = analysis_df[SOIL_CLASS_COLUMN].value_counts().index

# Create violin plots for each numerical parameter against the soil classes
plt.figure(figsize=(20, 10))
for i, col in enumerate(NUMERICAL_COLS, 1):
    plt.subplot(1, 3, i)
    sns.violinplot(data=analysis_df, x=SOIL_CLASS_COLUMN, y=col, order=class_order, cut=0)
    plt.title(f'Distribution of {col} by Soil Class', fontsize=14)
    plt.xlabel('Soil Class', fontsize=12)
    plt.ylabel(col, fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.yscale('log') # Log scale is essential for these parameters
    plt.grid(True, which="both", ls="--")

plt.suptitle('CPT Parameter Distributions Across Soil Classes', fontsize=20, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Take a random sample of the data to make the pair plot manageable
# Adjust the sample size based on your machine's performance
sample_size = 5000
sample_df = analysis_df.sample(n=min(sample_size, len(analysis_df)), random_state=42)

print(f"Creating a pair plot using a sample of {len(sample_df)} data points...")

# Create the pair plot
g = sns.pairplot(sample_df, hue=SOIL_CLASS_COLUMN, vars=NUMERICAL_COLS, 
                 plot_kws={'alpha': 0.6, 's': 20}, # Adjust point size and transparency
                 diag_kind='kde') # Use Kernel Density Estimate for diagonal plots

g.fig.suptitle('Pair Plot of CPT Parameters Colored by Soil Class (Sampled Data)', y=1.02)
plt.show()